In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from torchvision import models, transforms
from torch.autograd import Variable
from PIL import Image
import numpy as np
from path import Path as path
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
normalize = transforms.Normalize(
   mean=[0.485, 0.456, 0.406],
   std=[0.229, 0.224, 0.225]
)
preprocess = transforms.Compose([
   transforms.Scale(256),
   transforms.CenterCrop(224),
   transforms.ToTensor(),
   normalize
])

d:\Anaconda3\lib\site-packages\torchvision\transforms\transforms.py:188: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


In [4]:
imagenet = models.squeezenet1_1(pretrained=True)

d:\Anaconda3\lib\site-packages\torchvision\models\squeezenet.py:94: UserWarning: nn.init.kaiming_uniform is now deprecated in favor of nn.init.kaiming_uniform_.
  init.kaiming_uniform(m.weight.data)
d:\Anaconda3\lib\site-packages\torchvision\models\squeezenet.py:92: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  init.normal(m.weight.data, mean=0.0, std=0.01)


In [5]:
def PIL_to_raw(img_pil):
    img_tensor = preprocess(img_pil)
    img_tensor.unsqueeze_(0)
    img_variable = Variable(img_tensor)
    fc_out = imagenet(img_variable)
    return fc_out.data.numpy()

def pred_vector(img_pil):
    raw = PIL_to_raw(img_pil)
    preds = raw[0]
    result = np.array([np.exp(p) for p in preds])
    return result / sum(result)

In [6]:
mydir = "../../data/oxford-iiit-pet/"

In [7]:
def get_class(filename):
    parts = filename.split("_")
    return "_".join(parts[0:-1])

In [8]:
X, Y = [], []
for f in path(mydir + "/images").files():
    try:
        X.append(pred_vector(Image.open(f)))
        Y.append(get_class(f.name))
    except (OSError, RuntimeError):
        pass

In [9]:
Y2 = pd.get_dummies(Y)

In [10]:
Y2.head()

,Abyssinian,Bengal,Birman,Bombay,British_Shorthair,Egyptian_Mau,Maine_Coon,Persian,Ragdoll,Russian_Blue,...,newfoundland,pomeranian,pug,saint_bernard,samoyed,scottish_terrier,shiba_inu,staffordshire_bull_terrier,wheaten_terrier,yorkshire_terrier
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
X = np.array(X)

In [12]:
X.shape

(7378, 1000)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, Y2, test_size=0.2)

In [14]:
rf = RandomForestClassifier(n_estimators=50)

In [15]:
rf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [17]:
y_pred = rf.predict(X_test)

In [18]:
f1_score(y_test, y_pred, average='micro')

0.6415949252378794

In [19]:
accuracy_score(y_test, y_pred)

0.4796747967479675